# SVR, LR, RFのモデル作成

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
import GPy

EDAで作成したデータファイル（``data_7.csv``）から特徴量を読み込む

In [2]:
df_7 = pd.read_csv('data_7.csv')
df_7.describe()

,Id,age,domain1_var1,domain1_var2,domain2_var1,domain2_var2,IC_01,IC_07,IC_05,IC_16,...,CBN(13)_vs_DMN(94),CBN(18)_vs_DMN(94),CBN(4)_vs_DMN(94),CBN(7)_vs_DMN(94),CBN(18)_vs_CBN(13),CBN(4)_vs_CBN(13),CBN(7)_vs_CBN(13),CBN(4)_vs_CBN(18),CBN(7)_vs_CBN(18),CBN(7)_vs_CBN(4)
count,5877.000000,5877.000000,5877.000000,5877.000000,5877.000000,5877.000000,5877.000000,5877.000000,5877.000000,5877.000000,...,5877.000000,5877.000000,5877.000000,5877.000000,5877.000000,5877.000000,5877.000000,5877.000000,5877.000000,5877.000000
mean,15909.667007,50.034068,51.502462,59.304380,47.328355,51.910080,0.005368,0.009237,0.010620,0.000895,...,-0.126329,0.340097,0.126801,0.299151,0.495320,0.578637,0.461299,0.197640,0.768528,0.354929
std,3411.775315,13.539881,9.801768,10.957016,11.087953,11.799972,0.004585,0.004162,0.003571,0.003587,...,0.254345,0.180884,0.238343,0.201886,0.197955,0.268321,0.254933,0.299296,0.193878,0.182111
min,10001.000000,14.257265,15.769168,1.021874,0.991172,0.815285,-0.015894,-0.007958,-0.002240,-0.013459,...,-0.932657,-0.584421,-0.709769,-0.559527,-0.686442,-0.467751,-0.639171,-1.142909,-0.471138,-0.323693
25%,12961.000000,40.129361,45.397852,53.133474,40.225097,44.586221,0.002445,0.006437,0.008172,-0.001451,...,-0.302299,0.230182,-0.026810,0.173736,0.374122,0.424559,0.307198,0.022054,0.637085,0.243677
50%,15925.000000,50.427747,51.847306,60.052535,47.811205,52.572032,0.005512,0.009205,0.010567,0.000786,...,-0.111208,0.357885,0.146985,0.316375,0.497220,0.579708,0.476942,0.247063,0.755655,0.362175
75%,18886.000000,59.580851,57.892677,66.521451,55.024768,59.843566,0.008443,0.012035,0.012972,0.003207,...,0.061984,0.469599,0.300593,0.443144,0.615462,0.738983,0.624984,0.413300,0.881732,0.471903
max,21754.000000,84.491113,81.325580,94.702874,82.164478,94.509903,0.022888,0.027168,0.024085,0.022613,...,0.650448,0.852686,0.780373,0.912523,1.513935,2.123638,1.562309,1.102878,1.857374,1.282488


SVRや線形回帰を使用するので、特徴量および予測値お両者を標準化する。  
またfMRIデータ（相関係数）は数が多く、これに対する過学習を避けるため標準化したうえでさらに500で割り、特徴量の影響を軽減する。

In [36]:
X = df_7.iloc[:, 6:].values
y = df_7.iloc[:, 1:6].values

# train/validationに分割
X_train, X_val, y_train, y_val = train_test_split(
    X, y, random_state=1, train_size=0.75)

# 標準化
sc_x = StandardScaler()
sc_y = StandardScaler()
sc_x.fit(X_train)
sc_y.fit(y_train)
X_train_std = sc_x.transform(X_train)
X_val_std = sc_x.transform(X_val)
y_train_std = sc_y.transform(y_train)
y_val_std = sc_y.transform(y_val)

# fMRIデータ（86列目以降）は500で除す
X_train_tsf = X_train_std.copy()
X_train_tsf[:, 86:] = X_train_std[:, 86:]/500
X_val_tsf = X_val_std.copy()
X_val_tsf[:, 86:] = X_val_std[:, 86:]/500

まずsklearnのライブラリを使用し、各モデルについてGridSearchでハイパーパラメーターチューニングを行う。

### 1. SVR

In [61]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

# SVM
targets = df_7.columns[1:6]
svms = []
preds = np.empty_like(y_val)
for i, target in enumerate(targets):
    y_tr = y_train_std[:, i]
    
    parameters = {'C': [0.1, 1, 10]}
    model = GridSearchCV(SVR(), parameters, cv=5, return_train_score=False, iid=False)
    model.fit(X_train_tsf, y_tr)
    
    svms.append(model)
    preds[:, i] = model.predict(X_val_tsf)

preds = sc_y.inverse_transform(preds)
scores_svm = np.sum(np.abs(y_val - preds), axis=0) / np.sum(preds, axis=0)
print('C:', model.best_estimator_)
scores

 C:\Users\USER\anaconda3\envs\tf115\lib\site-packages\sklearn\model_selection\_search.py:823: FutureWarning:The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
 C:\Users\USER\anaconda3\envs\tf115\lib\site-packages\sklearn\model_selection\_search.py:823: FutureWarning:The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
 C:\Users\USER\anaconda3\envs\tf115\lib\site-packages\sklearn\model_selection\_search.py:823: FutureWarning:The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
 C:\Users\USER\anaconda3\envs\tf115\lib\site-packages\sklearn\model_selection\_search.py:823: FutureWarning:The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
 C:\Users\USER\anaconda3\envs\tf115\lib\site-packages\sklearn\model_selection\_search.py:823: FutureWarning:The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.


C: SVR(C=0.1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)


array([0.1555542 , 0.14754962, 0.14174629, 0.18465802, 0.17839163])

In [63]:
print('age')
print(svms[0].best_estimator_)
print('d1v1')
print(svms[1].best_estimator_)
print('d1v2')
print(svms[2].best_estimator_)
print('d2v1')
print(svms[3].best_estimator_)
print('d2v2')
print(svms[4].best_estimator_)

age
SVR(C=1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
d1v1
SVR(C=0.1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
d1v2
SVR(C=0.1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
d2v1
SVR(C=0.1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
d2v2
SVR(C=0.1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)


### 2.LR（Lasso, Ridge, ElasticNet）

In [58]:
from sklearn.linear_model import LassoCV

# Lasso
targets = df_7.columns[1:6]
lassos = []
preds = np.empty_like(y_val)

for i, target in enumerate(targets):
    y_tr = y_train_std[:, i]
    
    model = LassoCV(alphas=10 ** np.arange(-3, 1, 0.1), cv=5)
    model.fit(X_train_tsf, y_tr)
    
    lassos.append(model)
    preds[:, i] = model.predict(X_val_tsf)

preds = sc_y.inverse_transform(preds)
scores_l1 = np.sum(np.abs(y_val - preds), axis=0) / np.sum(preds, axis=0)
print(model)
print('α:', model.alpha_)
scores_l1

LassoCV(alphas=array([1.00000000e-03, 1.25892541e-03, 1.58489319e-03, 1.99526231e-03,
       2.51188643e-03, 3.16227766e-03, 3.98107171e-03, 5.01187234e-03,
       6.30957344e-03, 7.94328235e-03, 1.00000000e-02, 1.25892541e-02,
       1.58489319e-02, 1.99526231e-02, 2.51188643e-02, 3.16227766e-02,
       3.98107171e-02, 5.01187234e-02, 6.30957344e-02, 7.94328235e-02,
       1.00000000e-01, 1.25892541e-0...
       6.30957344e-01, 7.94328235e-01, 1.00000000e+00, 1.25892541e+00,
       1.58489319e+00, 1.99526231e+00, 2.51188643e+00, 3.16227766e+00,
       3.98107171e+00, 5.01187234e+00, 6.30957344e+00, 7.94328235e+00]),
        copy_X=True, cv=5, eps=0.001, fit_intercept=True, max_iter=1000,
        n_alphas=100, n_jobs=None, normalize=False, positive=False,
        precompute='auto', random_state=None, selection='cyclic', tol=0.0001,
        verbose=False)
α: 0.025118864315095874


array([0.15323069, 0.14579978, 0.13930254, 0.18202776, 0.17675102])

In [64]:
print('age')
print(lassos[0].alpha_)
print('d1v1')
print(lassos[1].alpha_)
print('d1v2')
print(lassos[2].alpha_)
print('d2v1')
print(lassos[3].alpha_)
print('d2v2')
print(lassos[4].alpha_)

age
0.0031622776601683824
d1v1
0.015848931924611172
d1v2
0.03162277660168389
d2v1
0.012589254117941701
d2v2
0.025118864315095874


In [59]:
from sklearn.linear_model import RidgeCV

# Ridge
targets = df_7.columns[1:6]
ridges = []
preds = np.empty_like(y_val)

for i, target in enumerate(targets):
    y_tr = y_train_std[:, i]
    
    model = RidgeCV(alphas=10 ** np.arange(-3, 1, 0.1), cv=5)
    model.fit(X_train_tsf, y_tr)
    
    ridges.append(model)
    preds[:, i] = model.predict(X_val_tsf)

preds = sc_y.inverse_transform(preds)
scores_l2 = np.sum(np.abs(y_val - preds), axis=0) / np.sum(preds, axis=0)
print(model)
print('α:', model.alpha_)
scores_l2

RidgeCV(alphas=array([1.00000000e-03, 1.25892541e-03, 1.58489319e-03, 1.99526231e-03,
       2.51188643e-03, 3.16227766e-03, 3.98107171e-03, 5.01187234e-03,
       6.30957344e-03, 7.94328235e-03, 1.00000000e-02, 1.25892541e-02,
       1.58489319e-02, 1.99526231e-02, 2.51188643e-02, 3.16227766e-02,
       3.98107171e-02, 5.01187234e-02, 6.30957344e-02, 7.94328235e-02,
       1.00000000e-01, 1.25892541e-0...-01, 1.99526231e-01,
       2.51188643e-01, 3.16227766e-01, 3.98107171e-01, 5.01187234e-01,
       6.30957344e-01, 7.94328235e-01, 1.00000000e+00, 1.25892541e+00,
       1.58489319e+00, 1.99526231e+00, 2.51188643e+00, 3.16227766e+00,
       3.98107171e+00, 5.01187234e+00, 6.30957344e+00, 7.94328235e+00]),
        cv=5, fit_intercept=True, gcv_mode=None, normalize=False, scoring=None,
        store_cv_values=False)
α: 0.10000000000000041


array([0.14296141, 0.14535155, 0.14057875, 0.18248216, 0.17639298])

In [65]:
print('age')
print(ridges[0].alpha_)
print('d1v1')
print(ridges[1].alpha_)
print('d1v2')
print(ridges[2].alpha_)
print('d2v1')
print(ridges[3].alpha_)
print('d2v2')
print(ridges[4].alpha_)

age
0.01995262314968885
d1v1
0.1995262314968889
d1v2
0.31622776601683955
d2v1
0.07943282347242846
d2v2
0.10000000000000041


In [60]:
from sklearn.linear_model import ElasticNetCV

# ElasticNet
targets = df_7.columns[1:6]
e_nets = []
preds = np.empty_like(y_val)

for i, target in enumerate(targets):
    y_tr = y_train_std[:, i]
    
    model = ElasticNetCV(l1_ratio=[0.1, 0.3, 0.5, 0.7, 0.9], cv=5)
    model.fit(X_train_tsf, y_tr)
    
    e_nets.append(model)
    preds[:, i] = model.predict(X_val_tsf)

preds = sc_y.inverse_transform(preds)
scores_l12 = np.sum(np.abs(y_val - preds), axis=0) / np.sum(preds, axis=0)
print(model)
print('l1/l2:', model.l1_ratio_)
scores_l12

ElasticNetCV(alphas=None, copy_X=True, cv=5, eps=0.001, fit_intercept=True,
             l1_ratio=[0.1, 0.3, 0.5, 0.7, 0.9], max_iter=1000, n_alphas=100,
             n_jobs=None, normalize=False, positive=False, precompute='auto',
             random_state=None, selection='cyclic', tol=0.0001, verbose=0)
l1/l2: 0.9


array([0.1532256 , 0.14580117, 0.13930923, 0.18191698, 0.17677925])

In [67]:
print('age')
print(e_nets[0].l1_ratio_)
print('d1v1')
print(e_nets[1].l1_ratio_)
print('d1v2')
print(e_nets[2].l1_ratio_)
print('d2v1')
print(e_nets[3].l1_ratio_)
print('d2v2')
print(e_nets[4].l1_ratio_)

age
0.9
d1v1
0.9
d1v2
0.3
d2v1
0.1
d2v2
0.9


次に、上記で求めたハイパーパラメーター設定下でクロスバリデーション学習を行う。  
K-foldライブラリを使用し、3通りのtrain/validationの分割で実施する。

### 1.SVR

In [7]:
from sklearn.model_selection import KFold

# SVR
targets = df_7.columns[1:6]
svms_7cv = []
preds = np.empty_like(y)

model_1 = SVR(C=1)
model_2 = SVR(C=0.1)
model_3 = SVR(C=0.1)
model_4 = SVR(C=0.1)
model_5 = SVR(C=0.1)
models = [model_1, model_2, model_3, model_4, model_5]

for i, target in enumerate(targets):
    y_cv = y_std[:, i]
    model = models[i]
    kfold = KFold(n_splits=3).split(X_std, y_cv)
    
    for train_index, test_index in kfold:
        X_train, X_val = X_std[train_index], X_std[test_index]
        y_train, y_val = y_cv[train_index], y_cv[test_index]
        model.fit(X_train, y_train)
    
    svms_7cv.append(model)
    preds[:, i] = model.predict(X_std)

preds = sc_y.inverse_transform(preds)
scores = np.sum(np.abs(y - preds), axis=0) / np.sum(preds, axis=0)
scores

array([0.09102709, 0.13642401, 0.13121689, 0.17008383, 0.16343184])

### 2. LR（Lasso, Ridge, ElasticNet）

In [8]:
from sklearn.linear_model import Lasso

# Lasso
targets = df_7.columns[1:6]
lassos_7cv = []
preds = np.empty_like(y)

model_1 = Lasso(alpha=0.0031622776601683824)
model_2 = Lasso(alpha=0.015848931924611172)
model_3 = Lasso(alpha=0.03162277660168389)
model_4 = Lasso(alpha=0.012589254117941701)
model_5 = Lasso(alpha=0.025118864315095874)
models = [model_1, model_2, model_3, model_4, model_5]

for i, target in enumerate(targets):
    y_cv = y_std[:, i]
    model = models[i]
    kfold = KFold(n_splits=3).split(X_std, y_cv)
    
    for train_index, test_index in kfold:
        X_train, X_val = X_std[train_index], X_std[test_index]
        y_train, y_val = y_cv[train_index], y_cv[test_index]
        model.fit(X_train, y_train)
    
    lassos_7cv.append(model)
    preds[:, i] = model.predict(X_std)

preds = sc_y.inverse_transform(preds)
scores = np.sum(np.abs(y - preds), axis=0) / np.sum(preds, axis=0)
scores

array([0.13219879, 0.14051894, 0.14044081, 0.17523369, 0.17329544])

In [10]:
from sklearn.linear_model import Ridge

# Ridge
targets = df_7.columns[1:6]
ridges_7cv = []
preds = np.empty_like(y)

model_1 = Ridge(alpha=0.01995262314968885)
model_2 = Ridge(alpha=0.1995262314968889)
model_3 = Ridge(alpha=0.31622776601683955)
model_4 = Ridge(alpha=0.07943282347242846)
model_5 = Ridge(alpha=0.10000000000000041)
models = [model_1, model_2, model_3, model_4, model_5]

for i, target in enumerate(targets):
    y_cv = y_std[:, i]
    model = models[i]
    kfold = KFold(n_splits=3).split(X_std, y_cv)
    
    for train_index, test_index in kfold:
        X_train, X_val = X_std[train_index], X_std[test_index]
        y_train, y_val = y_cv[train_index], y_cv[test_index]
        model.fit(X_train, y_train)
    
    ridges_7cv.append(model)
    preds[:, i] = model.predict(X_std)

preds = sc_y.inverse_transform(preds)
scores = np.sum(np.abs(y - preds), axis=0) / np.sum(preds, axis=0)
scores

array([0.13087478, 0.13639687, 0.1365768 , 0.17050615, 0.16568905])

In [72]:
# ElasticNet
targets = df_1.columns[1:6]
e_nets_1 = []
preds = np.empty_like(y_val)

for i, target in enumerate(targets):
    y_tr = y_train_std[:, i]
    
    model = ElasticNetCV(l1_ratio=[0.1, 0.3, 0.5, 0.7, 0.9], cv=5)
    model.fit(X_train_std, y_tr)
    
    e_nets_1.append(model)
    preds[:, i] = model.predict(X_val_std)

preds = sc_y.inverse_transform(preds)
scores_l12_1 = np.sum(np.abs(y_val - preds), axis=0) / np.sum(preds, axis=0)
print(model)
print('l1/l2:', model.l1_ratio_)
scores_l12_1

ElasticNetCV(alphas=None, copy_X=True, cv=5, eps=0.001, fit_intercept=True,
             l1_ratio=[0.1, 0.3, 0.5, 0.7, 0.9], max_iter=1000, n_alphas=100,
             n_jobs=None, normalize=False, positive=False, precompute='auto',
             random_state=None, selection='cyclic', tol=0.0001, verbose=0)
l1/l2: 0.1


array([0.15503568, 0.14625516, 0.13932186, 0.18215367, 0.17656677])

＜スコア一覧＞    
(data):      (age),    (d1_v1),    (d1_v2),    (d2_v1),    (d2_v2)   
svm   : 0.15555420, 0.14754962, 0.14174629, 0.18465802, 0.17839163  
lasso : 0.15323069, 0.14579978, 0.13930254, 0.18202776, 0.17675102  
ridge : 0.14296141, 0.14535155, 0.14057875, 0.18248216, 0.17639298  
e_net : 0.15322560, 0.14580117, 0.13930923, 0.18191698, 0.17677925  

SVR, LR（Lasso, Ridge）の保存  
ElasticNetはLassoとRidgeのブレンドであり、スコアも良いくないため使用しない

In [11]:
import pickle

# モデルを保存する
filename = 'svms_7cv.sav'
pickle.dump(svms_7cv, open(filename, 'wb'))

filename = 'lassos_7cv.sav'
pickle.dump(lassos_7cv, open(filename, 'wb'))

filename = 'ridges_7cv.sav'
pickle.dump(ridges_7cv, open(filename, 'wb'))

### 3. RandomForest
更にアンサンブル学習のバリエーションを増すため、Random Forestモデルも作成する。

In [8]:
from sklearn.ensemble import RandomForestRegressor

# ランダムフォレスト（深さは3に剪定）
forest = RandomForestRegressor(max_depth=3)
forests_7 = []
targets = df_7.columns[1:6]
preds = np.empty_like(y_val)

# 学習
for i, target in enumerate(targets):
    y_tr = y_train_std[:, i]
    forest.fit(X_train_std, y_tr)
    forests_7.append(forest)
    preds[:, i] = forest.predict(X_val_std)

＃
preds = sc_y.inverse_transform(preds)
score = np.sum(np.abs(y_val - preds), axis=0) / np.sum(preds, axis=0)
print(score)

# モデルの保存
import pickle
filename = 'forests_7.sav'
pickle.dump(forests_7, open(filename, 'wb'))

[0.16986683 0.13985699 0.13905636 0.18202676 0.17174111]
